In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import sys
sys.path.append('..')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
from models import LocalNoiseRBM

In [3]:
ds = tfds.load("mnist", with_info=False)

Instructions for updating:
Colocations handled automatically by placer.


W0310 17:16:40.187247 4567311808 deprecation.py:323] From /Users/btimar/anaconda3/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:423: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
dstr_all = ds['train']

trsize=10000
batch_size=128
epochs=10

dstr = dstr_all.take(trsize).shuffle(trsize).repeat(epochs).batch(batch_size)

In [5]:
feeds = dstr.make_one_shot_iterator().get_next()

In [6]:
trfeed = tf.reshape(feeds['image'], (-1, 784,1))

In [7]:
trfeed = tf.cast(trfeed, tf.float32)

In [8]:
nv = trfeed.shape[1]
nh = nv

In [9]:
k=1

In [10]:
rbm = LocalNoiseRBM(nv, nh)

In [11]:
grads = rbm.nll_gradients(trfeed,k, noise_condition=False)

In [12]:
samplesize=4*4
ksample=20
sample_feed = tf.placeholder(shape=(samplesize,nv,1),dtype=tf.float32)
sampler = rbm.build_sampler(sample_feed, ksample)

In [13]:
lr=.01

In [14]:
grads

[(<tf.Tensor 'Neg_6:0' shape=(1,) dtype=float32>,
  <tf.Variable 'noise_bias:0' shape=() dtype=float32_ref>),
 (<tf.Tensor 'Neg_7:0' shape=(784, 1) dtype=float32>,
  <tf.Variable 'hidden_bias:0' shape=(784, 1) dtype=float32_ref>),
 (<tf.Tensor 'Neg_8:0' shape=(784, 784) dtype=float32>,
  <tf.Variable 'weights:0' shape=(784, 784) dtype=float32_ref>),
 (<tf.Tensor 'Neg_9:0' shape=(784, 1) dtype=float32>,
  <tf.Variable 'visible_bias:0' shape=(784, 1) dtype=float32_ref>),
 (<tf.Tensor 'Neg_10:0' shape=(1,) dtype=float32>,
  <tf.Variable 'noise_kernel:0' shape=() dtype=float32_ref>)]

In [15]:
optimizer = tf.train.GradientDescentOptimizer(lr)
tr_op = optimizer.apply_gradients(grads)

ValueError: Shapes must be equal rank, but are 0 and 1 for 'GradientDescent/update_noise_bias/ApplyGradientDescent' (op: 'ApplyGradientDescent') with input shapes: [], [], [1].

In [ ]:
fw = tf.summary.FileWriter("logs", tf.get_default_graph())